In [7]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2

from lmfit import minimize, Minimizer, Parameters, fit_report
from easydict import EasyDict as edict
from typing import List

from lib.transformations import (euler_matrix,
                                 euler_from_matrix,
                                 scale_from_matrix,
                                 )
from lib.utils import convert_to_homogeneous, print_results
from lib.io import read_data_to_df
from lib.camera import Camera

CAM_NAMES = ('p1', 'p2')

cameras = edict()
for cam in CAM_NAMES:
    cameras[cam] = Camera(
            width=6000.,
            height=4000.,
            calib_path=f'../data/bundle/{cam}.txt'
        )

R1 = np.array(
    [[0.56935076,  0.81892662, -0.07210347],
    [-0.13614594,  0.00743009, -0.99066093],
    [-0.81074287,  0.57385015,  0.11572386]]
)
t1 = np.array(
    [-158., 108., 58.]
)
R2 = np.array(
    [[-0.19973149,  0.97983753,  0.00507472],
     [ 0.01583765,  0.00840668, -0.99983924],
     [-0.97972267, -0.19961901, -0.01719741], ]
)
t2 = np.array(
    [-235., 124., 363.] 
)

cameras.p1.R = R1
cameras.p1.t = t1
cameras.p2.R = R2
cameras.p2.t = t2


# read TPs
tie_points = edict()
for cam_id, cam in enumerate(cameras):
    tie_points[cam] = read_data_to_df(
        f'../data/bundle/keypoints_{cam}.txt',
        delimiter=',',
        header=0,
        col_names=['label', 'x', 'y'],
        index_col=0
    )
points3d_app = read_data_to_df(
    f'../data/bundle/points3d_app.txt',
    delimiter=',',
    header=0,
    col_names=['label', 'x', 'y'],
    index_col=0
)
num_pts = points3d_app.shape[0]


targets_image = read_data_to_df(
    '../data/space_resection/targets_image_IMG_2814.csv',
    delimiter=',',
    header=0,
    col_names=['label', 'x', 'y'],
    index_col=0
)


targets_world = read_data_to_df(
    '../data/space_resection/targets_world.csv',
    delimiter=',',
    header=0,
    col_names=['label', 'X', 'Y', 'Z'],
    index_col=0
)


Using OPENCV camera model + k3
Using OPENCV camera model + k3


In [8]:
def project_point(
    cam_prm,
    points,
    dist,
):
    
    rx, ry, rz = cam_prm['rx'], cam_prm['ry'], cam_prm['rz']
    tx, ty, tz = cam_prm['tx'], cam_prm['ty'], cam_prm['tz']
    fx, fy = cam_prm['fx'], cam_prm['fy']
    cx, cy = cam_prm['cx'], cam_prm['cy']
    
    # Build extrinsics and intrinsics matrix from parameters
    R = euler_matrix(rx, ry, rz)[:3,:3]
    t = np.array([tx, ty, tz])
    K = np.array([
        [fx, 0., cx],
        [0., fy, cy],
        [0., 0., 1.]
    ])
    
    # Project points and remove non-linear distortions
    rvec, _ = cv2.Rodrigues(R)
    projections, _ = cv2.projectPoints(
        np.expand_dims(points, 1),
        rvec, t,
        K, dist,
    )
    projections = projections[:, 0, :]
    
    return projections

# def compute__residuals():
#     pass


In [9]:
# Define parameters
params = Parameters()

for cam_id, cam in enumerate(cameras):
    rot = euler_from_matrix(cameras[cam].R)
    t = cameras[cam].t
    K = cameras[cam].K
    
    params.add(f'cam_{cam_id}_rx', value=rot[0], vary=True)
    params.add(f'cam_{cam_id}_ry', value=rot[1], vary=True)
    params.add(f'cam_{cam_id}_rz', value=rot[2], vary=True)
    params.add(f'cam_{cam_id}_tx', value=t[0], vary=True)
    params.add(f'cam_{cam_id}_ty', value=t[1], vary=True)
    params.add(f'cam_{cam_id}_tz', value=t[2], vary=True)
    params.add(f'cam_{cam_id}_fx', value=K[0, 0], vary=False)
    params.add(f'cam_{cam_id}_fy', value=K[1, 1], vary=False)
    params.add(f'cam_{cam_id}_cx', value=K[0, 2], vary=False)
    params.add(f'cam_{cam_id}_cy', value=K[1, 2], vary=False)

pts = points3d_app.to_numpy()
for i, pt in enumerate(pts):
    params.add(f'tps_{i:04}_X', value=pt[0], vary=True)
    params.add(f'tps_{i:04}_Y', value=pt[1], vary=True)
    params.add(f'tps_{i:04}_Z', value=pt[2], vary=True)

# print(params.pretty_print())    

In [11]:
# Compute residuals
def compute_residuals(
    params,
    cameras,
):
    
    residuals = list()

    for cam_id, cam in enumerate(cameras):

        tps_obs = tie_points[cam].to_numpy()
        dist = cameras[cam].dist

        parvals = params.valuesdict()

        cam_prm = {}
        cam_keys = ('rx', 'ry', 'rz', 'tx', 'ty', 'tz', 'fx', 'fy', 'cx', 'cy')
        parkeys = [f'cam_{cam_id}_{k}' for k in cam_keys]
        for k, pk in zip(cam_keys, parkeys):
            cam_prm[k] = parvals[pk]
        # print(cam_prm)


        for i in range(num_pts):
            tp_prm = [parvals[f'tps_{i:04}_{k}'] for k in ['X', 'Y', 'Z']]

            projection = project_point(
                cam_prm,
                tp_prm,
                dist
            )
            # print(projection)
            
            res = tps_obs[i] - projection
            # print(res)
            residuals.extend(res.flatten())

    residuals = np.array(residuals)
    # print(residuals)
    # print(residuals.shape)

    return residuals

# residuals = compute_residuals(params, cameras)
# print(residuals)

[-25.43285093   5.95165321 -29.09213544   6.17939624 -30.48491119
   6.74808597 -28.75684727   6.67400341 -29.47530255   6.72686775
 -29.51436297   6.29430987 -28.42123725   6.54753851 -30.40593446
   6.75236697 -26.91205249   6.06951867 -30.60059958   6.78779303
 -29.38421313   6.76054559 -29.47530255   6.72686775 -29.08472521
   6.17776606 -29.07962559   6.3714046  -30.48491119   6.74808597
 -29.58360924   6.31071965 -30.14910169   6.65381058 -24.73584273
   5.66962748 -28.21766332   6.34042155 -26.79926019   5.93356774
 -26.05323779   6.06318731 -26.38722228   5.83120718 -30.14910169
   6.65381058 -27.37552627   5.88706833 -24.90187513   5.68363715
 -29.07962559   6.3714046  -25.65470402   5.80667304 -29.09213544
   6.17939624 -26.38614761   5.97615638 -26.86137528   6.0753659
 -25.25768666   5.74464429 -31.70339371   6.80166558 -24.80271834
   5.66792002 -30.00604962   6.64458234 -26.46625413   5.99038742
 -26.95996947   5.85916243 -27.08501687   6.14004574 -30.37478166
   6.717549

In [17]:
# Run Optimization!
minimizer = Minimizer(
    compute_residuals,
    params,
    fcn_args=(
        cameras,
    ),
    # fcn_args=(
    #     targets_world.to_numpy(),
    #     targets_image.to_numpy(),
    # ),
    # fcn_kws={
    #     'dist': dist,
    #     'weights': weights,
    #     'prior_covariance_scale': sigma0_2,
    # },
    # scale_covar=False,
)
result = minimizer.minimize(method='leastsq')
print(fit_report(result))


[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 3291
    # data points      = 712
    # variables        = 546
    chi-square         = 1568.05735
    reduced chi-square = 9.44612862
    Akaike info crit   = 1654.13458
    Bayesian info crit = 4148.30512
##  Warning: uncertainties could not be estimated:
[[Variables]]
    cam_0_rx:    1.37159181 +/-        nan (nan%) (init = 1.371803)
    cam_0_ry:    0.92155769 +/- 840.704952 (91226.51%) (init = 0.94542)
    cam_0_rz:   -0.23728314 +/- 3219.54260 (1356835.80%) (init = -0.2347174)
    cam_0_tx:   -162.945960 +/- 1495123.76 (917558.05%) (init = -158)
    cam_0_ty:    109.365691 +/- 3146300.61 (2876862.56%) (init = 108)
    cam_0_tz:    59.5195508 +/- 892079.927 (1498801.51%) (init = 58)
    cam_0_fx:    6619.701 (fixed)
    cam_0_fy:    6619.701 (fixed)
    cam_0_cx:    3026.061 (fixed)
    cam_0_cy:    1889.983 (fixed)
    cam_1_rx:   -1.65627676 +/-        nan (nan%) (init = -1.656735)
    cam_1_ry:    1.4

OLD

In [ ]:
# targets_image = read_data_to_df(
#     '../data/space_resection/targets_image_IMG_2814.csv',
#     delimiter=',',
#     header=0,
#     col_names=['label', 'x', 'y'],
#     index_col=0
# )


# targets_world = read_data_to_df(
#     '../data/space_resection/targets_world.csv',
#     delimiter=',',
#     header=0,
#     col_names=['label', 'X', 'Y', 'Z'],
#     index_col=0
# )


# # A-priori Sigma_0²: scale of the covariance matrix
# sigma0_2 = 1.

# # Run Optimization!
# minimizer = Minimizer(
#     projection_residuals,
#     params,
#     fcn_args=(
#         targets_world.to_numpy(),
#         targets_image.to_numpy(),
#     ),
#     fcn_kws={
#         'dist': dist,
#         'weights': weights,
#         'prior_covariance_scale': sigma0_2,
#     },
#     scale_covar=False,
# )
# result = minimizer.minimize(method='leastsq')

# # Print result
# print_results(result, weights, sigma0_2)


In [ ]:
# # # from collections import namedtuple

# # # Camera_list = namedtuple('Camera_list', ['id', 'name'])
# # # # cams = {'p1': 1, 'p2': 2}
# # # cams = Camera_list('p1', 'p2')
# # # cams

# # from easydict import EasyDict as edict
# # cams = edict({'p1': 1, 'p2': 2})
# # cams[]

# from collections import UserList
# from lib.camera import Camera

# class CameraList(UserList):
    
#     def add_camera(self, camera: Camera):
#         self.append(camera)

# cams = CameraList(['p1', 'p1'])
